In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix 

## Import Data and prepare for analysis

In [3]:
df = pd.read_csv('melb_data.csv')

In [4]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [5]:
len(df.columns)

21

In [6]:
df.drop(['Propertycount','Address'],axis=1, inplace = True)#propertycount is equal to suburb, address is unique for each record, so we drop them
df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname
0,Abbotsford,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan
1,Abbotsford,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan
2,Abbotsford,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan
3,Abbotsford,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan
4,Abbotsford,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan


In [7]:
df['Suburb'].replace(' ','_',regex=True, inplace = True)
df['CouncilArea'].replace(' ','_',regex=True, inplace = True)
df['Regionname'].replace(' ','_',regex=True, inplace = True)
df.head(100) #replace whitespace with underscore 

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname
0,Abbotsford,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern_Metropolitan
1,Abbotsford,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern_Metropolitan
2,Abbotsford,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern_Metropolitan
3,Abbotsford,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern_Metropolitan
4,Abbotsford,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern_Metropolitan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Albert_Park,3,h,1370000.0,S,Greg,12/06/2016,3.3,3206.0,3.0,2.0,2.0,0.0,85.0,2000.0,Port_Phillip,-37.8451,144.9511,Southern_Metropolitan
96,Albert_Park,2,h,1000000.0,S,Cayzer,12/11/2016,3.3,3206.0,2.0,1.0,0.0,288.0,NaN,NaN,Port_Phillip,-37.8407,144.9523,Southern_Metropolitan
97,Albert_Park,3,h,2575000.0,S,Greg,15/10/2016,3.3,3206.0,3.0,2.0,1.0,228.0,NaN,NaN,Port_Phillip,-37.8445,144.9501,Southern_Metropolitan
98,Albert_Park,2,h,1322500.0,S,Greg,15/10/2016,3.3,3206.0,2.0,1.0,0.0,149.0,98.0,1890.0,Port_Phillip,-37.8443,144.9481,Southern_Metropolitan


In [8]:
df.drop(['Lattitude','Longtitude'],axis = 1, inplace = True) #since we already have  distance feature, which extracts from Lat,Long, so we can drop them.

In [9]:
df.dtypes

Suburb           object
Rooms             int64
Type             object
Price           float64
Method           object
SellerG          object
Date             object
Distance        float64
Postcode        float64
Bedroom2        float64
Bathroom        float64
Car             float64
Landsize        float64
BuildingArea    float64
YearBuilt       float64
CouncilArea      object
Regionname       object
dtype: object

we can see date is treated as object,we need to convert it into datetime and do feature engineering 

In [10]:
df['Date'] = pd.to_datetime(df['Date'],format='%d/%m/%Y')
df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname
0,Abbotsford,2,h,1480000.0,S,Biggin,2016-12-03,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,Northern_Metropolitan
1,Abbotsford,2,h,1035000.0,S,Biggin,2016-02-04,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,Northern_Metropolitan
2,Abbotsford,3,h,1465000.0,SP,Biggin,2017-03-04,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,Northern_Metropolitan
3,Abbotsford,3,h,850000.0,PI,Biggin,2017-03-04,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,Northern_Metropolitan
4,Abbotsford,4,h,1600000.0,VB,Nelson,2016-06-04,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,Northern_Metropolitan


In [11]:
df['year']=df['Date'].dt.year
df['month']=df['Date'].dt.month
df['day']=df['Date'].dt.day

In [12]:
df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname,year,month,day
0,Abbotsford,2,h,1480000.0,S,Biggin,2016-12-03,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,Northern_Metropolitan,2016,12,3
1,Abbotsford,2,h,1035000.0,S,Biggin,2016-02-04,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,Northern_Metropolitan,2016,2,4
2,Abbotsford,3,h,1465000.0,SP,Biggin,2017-03-04,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,Northern_Metropolitan,2017,3,4
3,Abbotsford,3,h,850000.0,PI,Biggin,2017-03-04,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,Northern_Metropolitan,2017,3,4
4,Abbotsford,4,h,1600000.0,VB,Nelson,2016-06-04,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,Northern_Metropolitan,2016,6,4


In [13]:
df.drop(['Date'],axis=1, inplace = True)

In [14]:
df.dtypes

Suburb           object
Rooms             int64
Type             object
Price           float64
Method           object
SellerG          object
Distance        float64
Postcode        float64
Bedroom2        float64
Bathroom        float64
Car             float64
Landsize        float64
BuildingArea    float64
YearBuilt       float64
CouncilArea      object
Regionname       object
year              int64
month             int64
day               int64
dtype: object

## dealing with Missing Data

In [15]:
df['Car'].unique() #nan exists in Car space

array([ 1.,  0.,  2.,  6.,  5.,  4.,  3.,  8.,  7.,  9., 10., nan])

In [16]:
df['Car'].isna().sum() #we may remove these records since it is a small portion 

62

In [17]:
df['YearBuilt'].unique() #nan exists in YearBuilt

array([  nan, 1900., 2014., 1910., 1890., 2005., 2009., 1880., 1985.,
       2010., 1970., 2013., 2012., 1920., 1950., 1998., 1930., 2016.,
       1965., 1980., 2011., 2015., 1975., 1993., 1960., 2003., 1995.,
       1996., 2000., 1925., 2004., 2008., 1962., 1955., 1988., 1953.,
       1997., 1990., 2007., 1967., 1964., 1889., 1888., 2006., 1934.,
       1935., 1966., 1969., 1940., 1938., 1887., 1904., 1972., 1978.,
       1939., 1945., 1947., 1958., 2017., 1999., 1992., 1961., 1928.,
       1977., 1976., 2001., 1948., 1927., 1981., 1943., 1974., 1968.,
       1957., 1987., 1986., 1926., 2002., 1994., 1949., 1905., 1912.,
       1924., 1952., 1991., 1942., 1973., 1915., 2018., 1916., 1982.,
       1971., 1963., 1897., 1989., 1919., 1923., 1983., 1903., 1830.,
       1895., 1937., 1908., 1913., 1917., 1885., 1902., 1854., 1850.,
       1898., 1941., 1951., 1946., 1956., 1984., 1901., 1959., 1979.,
       1933., 1886., 1929., 1906., 1870., 1932., 1875., 1863., 1918.,
       1893., 1856.,

In [18]:
df['YearBuilt'].isna().sum() #we may use average for each property type to fill the Nan

5375

In [19]:
df['CouncilArea'].unique()
df['CouncilArea'].astype(str) #nan exists in CouncilArea

0        Yarra
1        Yarra
2        Yarra
3        Yarra
4        Yarra
         ...  
13575      nan
13576      nan
13577      nan
13578      nan
13579      nan
Name: CouncilArea, Length: 13580, dtype: object

In [20]:
df['CouncilArea'].isna().sum() #probably remove them; or we can use KNN to predict the council area using other features then fill the missing value in

1369

In [21]:
df['BuildingArea'].unique() #nan exists in Building Area

array([         nan, 7.900000e+01, 1.500000e+02, 1.420000e+02,
       2.100000e+02, 1.070000e+02, 7.500000e+01, 1.900000e+02,
       9.400000e+01, 9.700000e+01, 1.100000e+02, 1.050000e+02,
       7.300000e+01, 1.350000e+02, 2.700000e+01, 6.000000e+01,
       8.500000e+01, 1.740000e+02, 5.000000e+01, 1.280000e+02,
       1.800000e+01, 1.240000e+02, 9.100000e+01, 8.200000e+01,
       8.100000e+01, 2.250000e+02, 1.450000e+02, 8.400000e+01,
       1.340000e+02, 1.160000e+02, 8.600000e+01, 1.270000e+02,
       1.860000e+02, 9.000000e+01, 1.330000e+02, 1.300000e+02,
       1.130000e+02, 1.000000e+02, 3.090000e+02, 1.920000e+02,
       1.260000e+02, 1.010000e+02, 1.020000e+02, 7.600000e+01,
       1.460000e+02, 1.980000e+02, 1.800000e+02, 1.590000e+02,
       1.220000e+02, 1.810000e+02, 9.800000e+01, 8.300000e+01,
       1.470000e+02, 2.540000e+02, 1.230000e+02, 1.310000e+02,
       3.400000e+01, 2.070000e+02, 6.800000e+01, 5.400000e+01,
       2.630000e+02, 2.030000e+02, 1.650000e+02, 1.4000

In [22]:
df['BuildingArea'].isna().sum() #probably fill in the average building area for its property type 

6450

Note: 
1. XGBOOST can automatically deal with null value in numerical features like YearBuilt and Building Area by applying average on stratified groups.  
2. we may construct two data set to test whether simply by removing the null value records gives a better testing result or by applying stratified average value.
3. or just removing all null value. we will go with this method first

In [23]:
df.dropna()

,Suburb,Rooms,Type,Price,Method,SellerG,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname,year,month,day
1,Abbotsford,2,h,1035000.0,S,Biggin,2.5,3067.0,2.0,1.0,0.0,156.0,79.00,1900.0,Yarra,Northern_Metropolitan,2016,2,4
2,Abbotsford,3,h,1465000.0,SP,Biggin,2.5,3067.0,3.0,2.0,0.0,134.0,150.00,1900.0,Yarra,Northern_Metropolitan,2017,3,4
4,Abbotsford,4,h,1600000.0,VB,Nelson,2.5,3067.0,3.0,1.0,2.0,120.0,142.00,2014.0,Yarra,Northern_Metropolitan,2016,6,4
6,Abbotsford,3,h,1876000.0,S,Nelson,2.5,3067.0,4.0,2.0,0.0,245.0,210.00,1910.0,Yarra,Northern_Metropolitan,2016,5,7
7,Abbotsford,2,h,1636000.0,S,Nelson,2.5,3067.0,2.0,1.0,2.0,256.0,107.00,1890.0,Yarra,Northern_Metropolitan,2016,10,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12205,Whittlesea,3,h,601000.0,S,Ray,35.5,3757.0,3.0,2.0,1.0,972.0,149.00,1996.0,Whittlesea,Northern_Victoria,2017,7,29
12206,Williamstown,3,h,1050000.0,VB,Williams,6.8,3016.0,3.0,1.0,0.0,179.0,115.00,1890.0,Hobsons_Bay,Western_Metropolitan,2017,7,29
12207,Williamstown,1,u,385000.0,SP,Williams,6.8,3016.0,1.0,1.0,1.0,0.0,35.64,1967.0,Hobsons_Bay,Western_Metropolitan,2017,7,29
12209,Windsor,2,u,560000.0,PI,hockingstuart,4.6,3181.0,2.0,1.0,1.0,0.0,61.60,2012.0,Stonnington,Southern_Metropolitan,2017,7,29


In [24]:
len(df)

13580

In [25]:
df.dropna(axis = 0,inplace = True)

In [26]:
df.isna().sum()

Suburb          0
Rooms           0
Type            0
Price           0
Method          0
SellerG         0
Distance        0
Postcode        0
Bedroom2        0
Bathroom        0
Car             0
Landsize        0
BuildingArea    0
YearBuilt       0
CouncilArea     0
Regionname      0
year            0
month           0
day             0
dtype: int64

## split data into dependent and independent variables

In [27]:
X = df.drop('Price',axis =1).copy()
X.head()

,Suburb,Rooms,Type,Method,SellerG,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname,year,month,day
1,Abbotsford,2,h,S,Biggin,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,Northern_Metropolitan,2016,2,4
2,Abbotsford,3,h,SP,Biggin,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,Northern_Metropolitan,2017,3,4
4,Abbotsford,4,h,VB,Nelson,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,Northern_Metropolitan,2016,6,4
6,Abbotsford,3,h,S,Nelson,2.5,3067.0,4.0,2.0,0.0,245.0,210.0,1910.0,Yarra,Northern_Metropolitan,2016,5,7
7,Abbotsford,2,h,S,Nelson,2.5,3067.0,2.0,1.0,2.0,256.0,107.0,1890.0,Yarra,Northern_Metropolitan,2016,10,8


In [28]:
y = df['Price'].copy()
y.head()

1    1035000.0
2    1465000.0
4    1600000.0
6    1876000.0
7    1636000.0
Name: Price, dtype: float64

## one-hot encoding

In [29]:
X.dtypes

Suburb           object
Rooms             int64
Type             object
Method           object
SellerG          object
Distance        float64
Postcode        float64
Bedroom2        float64
Bathroom        float64
Car             float64
Landsize        float64
BuildingArea    float64
YearBuilt       float64
CouncilArea      object
Regionname       object
year              int64
month             int64
day               int64
dtype: object

In [30]:
X_encoded = pd.get_dummies(X, columns = ['Suburb','Type','Method','SellerG','CouncilArea','Regionname'])
X_encoded.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,year,...,CouncilArea_Yarra,CouncilArea_Yarra_Ranges,Regionname_Eastern_Metropolitan,Regionname_Eastern_Victoria,Regionname_Northern_Metropolitan,Regionname_Northern_Victoria,Regionname_South-Eastern_Metropolitan,Regionname_Southern_Metropolitan,Regionname_Western_Metropolitan,Regionname_Western_Victoria
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,2016,...,1,0,0,0,1,0,0,0,0,0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,2017,...,1,0,0,0,1,0,0,0,0,0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,2016,...,1,0,0,0,1,0,0,0,0,0
6,3,2.5,3067.0,4.0,2.0,0.0,245.0,210.0,1910.0,2016,...,1,0,0,0,1,0,0,0,0,0
7,2,2.5,3067.0,2.0,1.0,2.0,256.0,107.0,1890.0,2016,...,1,0,0,0,1,0,0,0,0,0


## Build a preliminary XGBoost Model 

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y , random_state = 42)

In [32]:
xgb = XGBRegressor(base_score = 0.5, colsameple_byleve=1, colsample_bytree = 1, gamma = 0, learning_rate = 0.1, max_delta_step = 0,
                  max_depth=3,min_child_weight=1, missing=1, n_estimators=100, nthread=1, reg_alpha=0,
                  reg_lambda=1, scale_pos_weight=1,seed=1850,silent=True, subsample=1)

xgb.fit(X_train,y_train)


[21:07:12] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsameple_byleve=1,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=1,
             monotone_constraints='()', n_estimators=100, n_jobs=1, nthread=1,
             num_parallel_tree=1, predictor='auto', random_state=1850,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1850,
             silent=True, subsample=1, tree_method='exact', ...)

In [33]:
xgb.score(X_test,y_test) #default model gives score of 0.827

0.8274430232271993

## Optimize parameters using cross validation and gridsearch

In [213]:
## Round 1
param_test1 = {'max_depth':[3,4,5],
             'learning_rate':[0.1,0.01,0.05],
             'gamma':[0,0.25,1.0],
             'reg_lambda':[0,1.0,10.0],
             'scale_pos_weight':[1,3,5]}

In [215]:
grid_search_1 = GridSearchCV(estimator = XGBRegressor(base_score = 0.5, colsameple_byleve=1, colsample_bytree = 1, gamma = 0, learning_rate = 0.1, max_delta_step = 0,
                  max_depth=3,min_child_weight=1, missing=1, n_estimators=100, nthread=1, reg_alpha=0,
                  reg_lambda=1, scale_pos_weight=1,seed=1850,silent=True, subsample=1),param_grid = param_test1, scoring='r2', cv=5)

grid_search_1.fit(X_train,y_train)

[18:43:00] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:43:03] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:43:06] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false

[18:43:53] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:43:55] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:43:58] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false

[18:44:45] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:44:48] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:44:51] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false

[18:45:48] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:45:51] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:45:55] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false

[18:46:54] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:46:57] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:47:01] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false

...

[19:46:27] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:46:31] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:46:34] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false

[19:47:31] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:47:35] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:47:38] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false

[19:48:43] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:48:47] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:48:51] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false

[19:50:02] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:50:06] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:50:10] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=0.5, booster=None,
                                    colsameple_byleve=1, colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=1,
                                    enable_categorical=False, gamma=0,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=0.1, max_delta_step=0,
                                    max_depth=3, min_child_weight=1, missing=1,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, nthread=1,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=0,
                                    reg_lambda=1, scale_pos_weight=1, seed=1850,
                            

In [217]:
print(grid_search_1.best_params_)
print(grid_search_1.best_score_)

{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'reg_lambda': 1.0, 'scale_pos_weight': 1}
0.7983296953559677


In [34]:
#Round 2
param_test2 = {'max_depth':[5,7,9],
             'learning_rate':[0.1,0.5,1],
             'gamma':[0],
             'reg_lambda':[1.0],
             'scale_pos_weight':[0.1,0.5,1.0]}

In [35]:
grid_search_2 = GridSearchCV(estimator = XGBRegressor(base_score = 0.5, colsample_bytree = 1, gamma = 0, learning_rate = 0.1, max_delta_step = 0,
                  max_depth=5,min_child_weight=1, missing=1, n_estimators=100, nthread=1, reg_alpha=0,
                  reg_lambda=1, scale_pos_weight=1,seed=1850, subsample=1),param_grid = param_test2, scoring='r2', cv=5)

In [36]:
grid_search_2.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=0.5, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=1,
                                    enable_categorical=False, gamma=0,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=0.1, max_delta_step=0,
                                    max_depth=5, min_child_weight=1, missing=1,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, nthread=1,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=0,
                                    reg_lambda=1, scale_pos_weight=1, seed=1850,
                                    subsample=1, 

In [37]:
print(grid_search_2.best_params_)
print(grid_search_2.best_score_)

{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'reg_lambda': 1.0, 'scale_pos_weight': 0.1}
0.7983296953559677


parameter optimization finished.
{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'reg_lambda': 1.0, 'scale_pos_weight': 0.1}

## Building, Evaluating, drawing and interpreting the optimal XGBoost model


In [39]:
xgb = XGBRegressor(base_score = 0.5, colsameple_byleve=1, colsample_bytree = 1, gamma = 0, learning_rate = 0.1, max_delta_step = 0,
                  max_depth=5,min_child_weight=1, missing=1, n_estimators=100, nthread=1, reg_alpha=0,
                  reg_lambda=1, scale_pos_weight=0.1,seed=1850,silent=True, subsample=1)

xgb.fit(X_train,y_train)

[21:36:07] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "colsameple_byleve", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsameple_byleve=1,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=1,
             monotone_constraints='()', n_estimators=100, n_jobs=1, nthread=1,
             num_parallel_tree=1, predictor='auto', random_state=1850,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=0.1, seed=1850,
             silent=True, subsample=1, tree_method='exact', ...)

In [42]:
print ("Validation:",xgb.score(X_test,y_test)) #score increased by 0.02 than default paramters

Validation: 0.844636144009966


In [43]:
predict = xgb.predict(X_test)